## Election Generator

This notebook generates fake elections that simulate the racial dynamics of an election, at the precinct (or lowest cohesive voting bloc) level.

Given:
1. the number of people in the precinct: $n$
2. the list of candidates in the district: $C = \{c_0, c_1, \dots, c_a\}$, where $a$ is the number of candidates in the race
3. the racial breakdown of the district: $R = \{r_0, r_1, \dots, r_b\}$, where $b$ is the number of races represented. $r_i$ is the fraction of $n$ that race $i$ represents in the district
4. the probabilities that members of certain races vote for certain candidates: $\beta_{i, c_j}$, where $i$ is the race and $c_j$ is the candidate

The election generated has the following information:
1. the number of people that voted for each candidate
2. the racial breakdown of each candidate's voters

In [46]:
from typing import Dict, List, Tuple
import random
from scipy.stats import binom
import numpy as np

In [21]:
def generate_deterministic_election(n: int, candidates: List[str], racial_breakdown: List[float], beta: List[List[float]]) -> Dict[str, Tuple[int, float]]:
    racial_numbers = [round(r_i * n) for r_i in racial_breakdown]
    result = dict()
    for cand_index, candidate in enumerate(candidates):
        racial_result = [r_n * beta[race][cand_index] for race, r_n in enumerate(racial_numbers)]
        num_votes = sum(racial_result)
        result[candidate] = num_votes, racial_result
    return result

In [56]:
generate_deterministic_election(1000, ['a', 'b', 'c'], [0.6, 0.3, 0.1], [[0.4, 0.5, 0.1], [0.3, 0.2, 0.5], [0.3, 0.5, 0.2]])

[600, 300, 100]


{'a': (360.0, [240.0, 90.0, 30.0]),
 'b': (410.0, [300.0, 60.0, 50.0]),
 'c': (230.0, [60.0, 150.0, 20.0])}

In [48]:
def generate_random_election(n: int, candidates: List[str], racial_breakdown: List[float], beta: List[List[float]], whole_output=False) -> Dict[str, Tuple[int, float]]:
    racial_numbers = [round(r_i * n) for r_i in racial_breakdown]
    result = dict()
    for cand_index, candidate in enumerate(candidates):
        racial_result = [binom.rvs(n=r_n, p=beta[race][cand_index], size=100).mean() for race, r_n in enumerate(racial_numbers)]
        num_votes = sum(racial_result)
        if whole_output:
            num_votes = round(num_votes)
            racial_result = np.round(racial_result).tolist()
        result[candidate] = num_votes, racial_result
    return result

In [57]:
generate_random_election(1000, ['a', 'b', 'c'], [0.6, 0.3, 0.1], [[0.4, 0.5, 0.1], [0.3, 0.2, 0.5], [0.3, 0.5, 0.2]])

{'a': (359.53999999999996, [239.41, 90.41, 29.72]),
 'b': (409.56000000000006, [299.56, 59.78, 50.22]),
 'c': (227.31, [58.57, 149.19, 19.55])}

In [58]:
generate_random_election(1000, ['a', 'b', 'c'], [0.6, 0.3, 0.1], [[0.4, 0.5, 0.1], [0.3, 0.2, 0.5], [0.3, 0.5, 0.2]], whole_output=True)

{'a': (360.0, [240.0, 90.0, 30.0]),
 'b': (411.0, [301.0, 61.0, 50.0]),
 'c': (227.0, [59.0, 148.0, 20.0])}